### Загружаем данные

* Данные по кампаниям за 1 год
* **165823** кампаний
* **29** параметров
* Параметры рассчитывались за **0,1,2 и 3 дни** жизни пользователя
* Нужно прогнозировать **CARPU 60 дня**

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('data_users.csv',delimiter =',')
data.head()

In [ ]:
data.describe()

### Поработаем с данными
* Заполним NaN
* Нормализуем значения параметров
* Промасштабируем значения параметров

In [ ]:
#Сколько у нас пропущенных значений?
missing = (data.isnull().sum() / len(data)) * 100
missing = missing.drop(missing[missing==0].index).sort_values(ascending=False)
missing = pd.DataFrame({'Missing Ratio': missing})
missing.head(20)

In [ ]:
# Заполним пропущенные значения и проверим
data.fillna(0, inplace = True)

missing = (data.isnull().sum() / len(data)) * 100
missing = missing.drop(missing[missing==0].index).sort_values(ascending=False)
missing = pd.DataFrame({'Missing Ratio': missing})
missing.head()

In [ ]:
# Зададим X и y
X = data.iloc[:,2:31]
y = data['y_target']
y.describe()

In [ ]:
# Нормализуем данные и смотрим коэффициент ассиметрии
from scipy.special import boxcox1p
from scipy.stats import skew
import numpy as np

numeric_feats = X.dtypes[X.dtypes!="object"].index
skewed_feats = X[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending = False)
skewness = pd.DataFrame({'Skew':skewed_feats})
skewness = skewness[abs(skewness)>0.2]
skewed_features = skewness.index
lam=0.04
for feat in skewed_features:
    X[feat] = boxcox1p(X[feat],lam)

# Логарифмируем target и смотрим коэффициент ассиметрии
y_n=boxcox1p(y,0)
print("Коэффициент ассиметрии y до транфсормации – %s и после – %s"% (skew(y),skew(y_n)))

In [ ]:
y_n.describe()

In [ ]:
# Масштабируем с Robust Scaler-ом
from sklearn.preprocessing import RobustScaler
sc = RobustScaler()
names = list(X.columns)
X= sc.fit_transform(X)
X = pd.DataFrame(data=X, columns=names)
X.fillna(0, inplace = True)

In [ ]:
X.head()

### Перейдем к моделированию
* **Модель:** Будем использовать Ridge regression
* **Кросс-валидация**: алгоритм ShuffleSplit
* **Работа с параметрами:** Pearson score, RFECV
* **Регуляризация**. Посмотрим как выбирается параметр Alpha
* **Learning Curve**: Посмотрим как ведет себя модель на обучающей и валидирующей выборках
* **Residuals Plot**: Посмотрим как ведут себя остатки (разница между реальным и пронозным значениями)
* **Prediction Error Plot**: Сравним прогнозные результаты модели с ее реальными значениями

In [ ]:
from sklearn.linear_model import  RidgeCV
from sklearn.model_selection import ShuffleSplit, train_test_split

cv=ShuffleSplit(n_splits=4,train_size=0.8,test_size=0.2,random_state= 2019)
regressor = RidgeCV()

### Смотрим R2 и MSE

In [ ]:
import matplotlib.pyplot as plt
from yellowbrick.model_selection import CVScores

_, ax = plt.subplots(figsize=(10,10))
r2 = CVScores(regressor,ax=ax,cv=cv,scoring='r2')
r2.fit(X,y_n)
r2.poof()

In [ ]:
_, ax = plt.subplots(figsize=(10,10))
mse = CVScores(regressor,ax=ax,cv=cv,scoring='neg_mean_squared_error')
mse.fit(X,y_n)
mse.poof()

In [ ]:
from sklearn.metrics import make_scorer

def rmse(y, y_pred):
    return np.sqrt(np.mean((y_pred - y)**2))
scorer = make_scorer(rmse,greater_is_better=False)
_, ax = plt.subplots(figsize=(10,10))
mse = CVScores(regressor,ax=ax,cv=cv,scoring=scorer)
mse.fit(X,y_n)
mse.poof()

In [ ]:
y_n.describe()

### Поработаем с параметрами

In [ ]:
# Посмотрим коллеряционную матрицу
from yellowbrick.features import Rank2D,RFECV
_, ax = plt.subplots(figsize=(20,10))
rank = Rank2D(ax=ax,features=names,algorithm='pearson')
rank.fit(X,y_n)
rank.transform(X)
rank.poof()

In [ ]:
%%time
# Попробуем посмотреть Recursive Feature Elimination
_, ax = plt.subplots(figsize=(20,10))
rfe = RFECV(regressor,cv=cv,scoring=scorer,step=15)
rfe.fit(X,y_n)
rfe.poof()

In [ ]:
#Сохраним выбранные параметры
params = {
    'parameters': list(X.columns),
    'ranking':list(rfe.ranking_)
}
par = pd.DataFrame(data=params)
huber_params = list(par[par['ranking']==1]['parameters'])

In [ ]:
# Посмотрим коллеряционную матрицу еще раз
_, ax = plt.subplots(figsize=(20,10))
rank = Rank2D(ax=ax,features=huber_params,algorithm='pearson')
rank.fit(X[huber_params],y_n)
rank.transform(X[huber_params])
rank.poof()

### Теперь поработаем с регуляризационным коэффициентом alpha

In [ ]:
from yellowbrick.regressor import AlphaSelection
_, ax = plt.subplots(figsize=(20,10))
alphas = np.logspace(-10, 1, 400)
a_select=AlphaSelection(RidgeCV(alphas=alphas))
a_select.fit(X[huber_params],y_n)
a_select.poof()

### Посмотрим Learning Curve

In [ ]:
%%time
from yellowbrick.model_selection import LearningCurve
sizes = np.linspace(0.2, 1.0, 500)
_, ax = plt.subplots(figsize=(20,10))
lc = LearningCurve(RidgeCV(alphas=alphas), scoring=scorer,cv=cv)
lc.fit(X[huber_params],y_n)
lc.poof()

In [ ]:
_, ax = plt.subplots(figsize=(20,10))
lc = LearningCurve(RidgeCV(alphas=alphas), scoring='r2',cv=cv)
lc.fit(X[huber_params],y_n)
lc.poof()

### Посмотрим, что нам дает обученная модель

In [ ]:
# Смотрим как ведут себя остатки
from sklearn.model_selection import train_test_split
from yellowbrick.regressor import ResidualsPlot,PredictionError
X_train,X_test,y_train,y_test = train_test_split(X[huber_params],y_n,test_size=0.2,random_state=2019)
regressor=RidgeCV(alphas=alphas,cv=cv,scoring=scorer)

_, ax = plt.subplots(figsize=(20,10))
res = ResidualsPlot(regressor)
res.fit(X_train,y_train)
res.score(X_test,y_test)
res.poof()

In [ ]:
# Смотрим как распределены ошибки
_, ax = plt.subplots(figsize=(20,10))
error=PredictionError(regressor)
error.fit(X_train,y_train)
error.score(X_test,y_test)
error.poof()